<a href="https://colab.research.google.com/github/aushinanna/chatbot/blob/master/BERT%20QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install transformers

     |████████████████████████████████| 645kB 2.7MB/s 
     |████████████████████████████████| 1.1MB 13.5MB/s 
     |████████████████████████████████| 890kB 17.2MB/s 
     |████████████████████████████████| 3.8MB 23.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=ca734e05c46ce4a6abd9212e68ab98817adbcf4f91d7f71c4b020d1e1c00ac9e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
import torch

In [0]:
from transformers import BertForQuestionAnswering
model=BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [0]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [0]:
question="How many parameters does BERT-large have?"
answer_text="BERT-large is really big...it has 24 layers and an embedding size of 1,024, for a total of 340M prameters! Altogether its 1.34GB,so expect it to take a couple minutes to download to your colab instance. "

In [0]:
input_ids=tokenizer.encode(question,answer_text)
print('The total number of tokens is {:}'.format(len(input_ids)))

The total number of tokens is 70


In [0]:
tokens=tokenizer.convert_ids_to_tokens(input_ids)
for token,id in zip(tokens,input_ids):
  if id==tokenizer.sep_token_id:
    print('')
  print('{:<12},{:>6}'.format(token,id))
  if id==tokenizer.sep_token_id:
    print('')

[CLS]       ,   101
how         ,  2129
many        ,  2116
parameters  , 11709
does        ,  2515
bert        , 14324
-           ,  1011
large       ,  2312
have        ,  2031
?           ,  1029

[SEP]       ,   102

bert        , 14324
-           ,  1011
large       ,  2312
is          ,  2003
really      ,  2428
big         ,  2502
.           ,  1012
.           ,  1012
.           ,  1012
it          ,  2009
has         ,  2038
24          ,  2484
layers      ,  9014
and         ,  1998
an          ,  2019
em          ,  7861
##bed       ,  8270
##ding      ,  4667
size        ,  2946
of          ,  1997
1           ,  1015
,           ,  1010
02          ,  6185
##4         ,  2549
,           ,  1010
for         ,  2005
a           ,  1037
total       ,  2561
of          ,  1997
340         , 16029
##m         ,  2213
pr          , 10975
##ame       , 14074
##ters      ,  7747
!           ,   999
altogether  , 10462
its         ,  2049
1           ,  1015
.           ,  101

In [0]:
#Please check this plart
sep_index=input_ids.index(tokenizer.sep_token_id)
num_seg_a=sep_index + 1
num_seg_b=len(input_ids)- num_seg_a
segment_ids=[0]*num_seg_a + [1] *num_seg_b
assert len(segment_ids)==len(input_ids)

In [0]:
start_scores , end_scores =model(torch.tensor([input_ids]),token_type_ids=torch.tensor([segment_ids]))

In [0]:
# Run our example through the model.
start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                 token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text


In [0]:
answer_start=torch.argmax(start_scores)   ##error portion
answer_end=torch.argmax(end_scores)
answer=''.join(tokens[answer_start:answer_end +1])
print('Answer:"' +answer+ '"')

Answer:"24"


In [0]:
answer_start

tensor(22, grad_fn=<NotImplemented>)

In [0]:
start_scores

tensor([[-6.3752, -6.9381, -8.1985, -8.7307, -7.9238, -8.2095, -8.5854, -8.6564,
         -7.8564, -9.3947, -6.3752, -2.2245, -6.3341, -5.6846, -7.8555, -7.1270,
         -6.5383, -7.2298, -7.1630, -7.3402, -0.2914, -3.6906,  4.0670, -2.9281,
         -6.0480, -3.4837, -3.8319, -5.7394, -6.3772, -4.3910, -5.6930,  0.8205,
         -5.5484, -5.0738, -4.4025, -6.1734, -2.7641, -2.2271, -1.8825, -5.3404,
          1.7396, -4.9956, -4.0568, -6.6698, -5.2469, -5.9126, -3.2724, -4.2475,
         -0.5633, -6.9760, -3.9515, -5.9604, -8.3055, -7.0333, -6.2999, -7.0149,
         -8.1446, -7.0122, -7.7905, -6.8471, -7.1210, -8.4932, -6.4802, -8.1920,
         -7.6449, -5.8207, -8.1993, -6.6946, -8.2576, -6.3752]],
       grad_fn=<SqueezeBackward1>)

In [0]:
answer=tokens[answer_start]
for i in range (answer_start +1,answer_end+1):
  if tokens[i][0:2] =='##':
    answer+=tokens[i][2:]
  else:
      answer+=''+tokens[i]
      print('Answer:"' +answer+ '"')

